In [522]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 2000
pd.options.display.max_columns = 2000
from sklearn.utils import resample

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, \
AdaBoostClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier

Read data:

In [523]:
path = '../../data/'

In [524]:
df_train = pd.read_csv(path+'application_train.csv')
# df_cc_bal = pd.read_csv(path+'credit_card_balance.csv')
df_pos = pd.read_csv(path+'POS_CASH_balance.csv')

In [525]:
df_train.shape

(307511, 122)

In [526]:
df_train.groupby(by='TARGET')['SK_ID_CURR'].nunique()

TARGET
0    282686
1     24825
Name: SK_ID_CURR, dtype: int64

In [527]:
df_majority = df_train[df_train.TARGET==0]
df_minority = df_train[df_train.TARGET==1]

In [528]:
# downsample majority class
df_majority_downsampled = resample(df_majority,replace=True,\
                                  n_samples=len(df_minority),\
                                  random_state=23)

df_train = pd.concat([df_majority_downsampled,df_minority])

In [529]:
# # upsample minority class
# df_majority_upsampled = resample(df_minority,replace=True,\
#                                   n_samples=len(df_majority),\
#                                   random_state=23)

# df_train = pd.concat([df_majority_upsampled,df_majority])

In [530]:
df_train.shape

(49650, 122)

In [531]:
df_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
139035,261215,0,Cash loans,F,N,N,0,135000.0,622413.0,33763.5,495000.0,Family,Working,Secondary / secondary special,Married,Municipal apartment,0.031329,-14257,-375,-4308.0,-4420,NaN,1,1,1,1,1,0,NaN,2.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Military,0.458895,0.550935,0.506484,0.1052,0.0761,0.9781,NaN,NaN,0.08,0.0690,0.3333,NaN,0.0836,NaN,0.1263,NaN,0.0038,0.1071,0.079,0.9782,NaN,NaN,0.0806,0.0690,0.3333,NaN,0.0855,NaN,0.1316,NaN,0.0041,0.1062,0.0761,0.9781,NaN,NaN,0.08,0.0690,0.3333,NaN,0.0851,NaN,0.1286,NaN,0.0039,NaN,block of flats,0.1002,"Stone, brick",No,3.0,0.0,3.0,0.0,-1931.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0
295266,442083,0,Cash loans,M,N,Y,0,117000.0,247500.0,15084.0,247500.0,Family,Working,Secondary / secondary special,Married,House / apartment,0.028663,-21890,-2682,-5069.0,-4243,NaN,1,1,0,1,1,0,Laborers,2.0,2,2,TUESDAY,16,0,0,0,0,0,0,Industry: type 9,NaN,0.511710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-791.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
242842,381096,0,Revolving loans,F,N,N,0,202500.0,180000.0,9000.0,180000.0,Family,State servant,Higher education,Married,House / apartment,0.004960,-12084,-1879,-1548.0,-4653,NaN,1,1,0,1,0,0,NaN,2.0,2,2,THURSDAY,13,0,0,0,0,0,0,Government,NaN,0.307972,0.629674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,-30.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
295745,442639,0,Cash loans,F,N,N,0,243000.0,585000.0,28273.5,585000.0,Unaccompan

Feature engineering:

Before any feature engineering, ROCAUC:

In [532]:
0.75487777950760337

0.7548777795076034

In [533]:
AMT_cols = ['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE']

for i in range(len(AMT_cols)):
    for j in range(len(AMT_cols)):
        if i != j:
            col_1 = AMT_cols[i]
            col_2 = AMT_cols[j]
            df_train[col_1+'_OVER_'+col_2] = 1.0 * df_train[col_1] / df_train[col_2]

ROCAUC:

In [534]:
0.76034363338930977

0.7603436333893098

In [535]:
control_for_age_cols = ['DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH']

for col in control_for_age_cols:
    df_train[col+'_OVER_DAYS_BIRTH'] = 1.0 * df_train[col] / df_train.DAYS_BIRTH

ROCAUC:

In [536]:
0.76111507043858218

0.7611150704385822

In [537]:
df_train['CNT_FAM_MEMBERS_OVER_DAYS_BIRTH'] = 1.0 * df_train.CNT_FAM_MEMBERS / (-df_train.DAYS_BIRTH)

ROCAUC:

In [538]:
0.76153678901265509

0.7615367890126551

Adding AdaBoost to Voting Classifier, ROCAUC:

In [539]:
0.76158009812369776

0.7615800981236978

In [540]:
# df_cc_bal.head()

In [541]:
# df_cc_bal = df_cc_bal[[x for x in df_cc_bal.columns if x not in ['SK_ID_PREV']]]

In [542]:
# df_cc_bal_active = df_cc_bal.loc[df_cc_bal.NAME_CONTRACT_STATUS == 'Active', :]

In [543]:
# df_cc_bal_active_mean = df_cc_bal_active.groupby(by='SK_ID_CURR').mean()
# df_cc_bal_active_median = df_cc_bal_active.groupby(by='SK_ID_CURR').median()
# df_cc_bal_active_std = df_cc_bal_active.groupby(by='SK_ID_CURR').std()

In [544]:
# df_cc_bal_active_mean.shape, df_cc_bal_active_median.shape, df_cc_bal_active_std.shape

In [545]:
# df_cc_bal_active_agg = pd.concat([df_cc_bal_active_mean,df_cc_bal_active_median,df_cc_bal_active_std])

In [546]:
# df_cc_bal_active_agg.reset_index(inplace=True)

In [547]:
# df_all = df_train.join(other=df_cc_bal_active_agg,how='left',on='SK_ID_CURR',rsuffix='_copy')
# del(df_all['SK_ID_CURR_copy'])

ROCAUC:

In [548]:
0.7613472727048014

0.7613472727048014

In [549]:
df_pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [550]:
df_pos = df_pos[[x for x in df_pos.columns if x not in ['SK_ID_PREV']]]

In [551]:
df_pos_median = df_pos.groupby(by='SK_ID_CURR').median()
df_pos_std = df_pos.groupby(by='SK_ID_CURR').std()

df_pos_agg = pd.concat([df_pos_median,df_pos_std])

In [552]:
df_pos.reset_index(inplace=True)

In [553]:
df_all = df_train.join(other=df_pos,how='left',on='SK_ID_CURR',rsuffix='_copy')
df_all[[x for x in df_all.columns if x != 'SK_ID_CURR_copy']]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [554]:
df_all.shape

(49650, 146)

ROCAUC (admittedly i changed filling na to from 0 to med here)

Model:

In [555]:
X = df_all[[x for x in df_all.columns if x not in ['SK_ID_CURR','TARGET']]]
y = df_all['TARGET']

In [556]:
X.shape

(49650, 144)

In [557]:
X = X.fillna('med')

In [558]:
X = pd.get_dummies(X, dummy_na = True, drop_first = True)

In [559]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, \
                                                   random_state = 23)

Voting Classifier:

In [560]:
# # kfold = model_selection.KFold(n_splits=10, random_state=23)

# estimators = []
# model1 = GradientBoostingClassifier(random_state = 23)
# estimators.append(('one', model1))
# model2 = AdaBoostClassifier(random_state = 23)

# clf = VotingClassifier(estimators, voting = 'soft')

In [ ]:
clf = GradientBoostingClassifier(random_state = 23)

In [ ]:
y_pred = clf.fit(X_train, y_train).predict_proba(X_test)

In [ ]:
y_pred = [y[1] for y in y_pred]

In [ ]:
roc_auc_score(y_test,y_pred)

In [ ]:
X.shape